In [1]:
import os
import pandas as pd
import numpy as np
import yaml
from tqdm import tqdm

#セットアップ
tqdm.pandas()

import sys
sys.path.append("/kaggle/src")
from kaggle_metrics.kaggle_kl_div import compute_each_score

In [8]:
LB_EXPERTNUM_THR_LOW = 5
LB_EXPERTNUM_THR_HIGH = 15

In [2]:
model1_oof_path = "/kaggle/working/exp003_resnet34d/oof_with_score.csv"
model2_oof_path = "/kaggle/working/exp004_effnet_b0_ns/oof_with_score.csv"
model3_oof_path = "/kaggle/working/exp004_effnet_b1_ns/oof_with_score.csv"

model1_oof = pd.read_csv(model1_oof_path)
model2_oof = pd.read_csv(model2_oof_path)
model3_oof = pd.read_csv(model3_oof_path)

In [30]:
model1_oof["score"].mean(), model2_oof["score"].mean(), model3_oof["score"].mean()

(0.7236865891754584, 0.6636904705550419, 0.6331114417486615)

In [13]:
ens_oof = model1_oof.copy()
pred_cols = [col for col in ens_oof.columns if "pred" in col]
ens_oof[pred_cols] = (model1_oof[pred_cols] + model2_oof[pred_cols] + model3_oof[pred_cols]) / 3
ens_oof[pred_cols] = ens_oof[pred_cols] / ens_oof[pred_cols].values.sum(axis=1, keepdims=True)

In [14]:
ens_oof["score"] = ens_oof.progress_apply(compute_each_score, axis=1)

100%|██████████| 12948/12948 [02:05<00:00, 103.52it/s]


In [15]:
ens_oof["score"].mean()

1.1195751656581954

In [16]:
display(ens_oof.groupby("expert_consensus")["score"].mean())

expert_consensus
GPD        1.289953
GRDA       1.655591
LPD        1.332616
LRDA       1.882303
Other      0.726965
Seizure    1.197654
Name: score, dtype: float64

In [29]:
ens_oof

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,...,total_evaluators,target,fold,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote,score
0,1202099836,0,0.0,1353070,0,0.0,3051612434,34554,Other,0.0,...,14,Other,0,0.055458,0.059976,0.157455,0.033681,0.121492,0.571937,0.367644
1,1445780287,0,0.0,4004824,0,0.0,3042959589,22597,Other,0.0,...,17,Other,0,0.550663,0.112090,0.210461,0.011898,0.025287,0.089602,2.412380
2,3669581453,0,0.0,4546138,0,0.0,3616273655,57448,LRDA,0.0,...,3,LRDA,0,0.303979,0.043134,0.061732,0.015805,0.034121,0.541230,4.147447
3,3100586988,0,0.0,4960933,0,0.0,1900116647,35590,Other,0.0,...,2,Other,0,0.028856,0.045082,0.043460,0.024656,0.056355,0.801591,0.221156
4,2559567335,0,0.0,5487370,0,0.0,1418039992,36059,Other,0.0,...,12,Other,0,0.071127,0.024948,0.009986,0.090556,0.329151,0.474232,0.746059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12943,3570444920,0,0.0,2143352195,0,0.0,3891513135,31417,GRDA,0.0,...,3,GRDA,4,0.049894,0.059990,0.069197,0.138316,0.406295,0.276308,0.900675
12944,3897272486,0,0.0,2143410921,0,0.0,1360814571,63946,Other,0.0,...,15,Other,4,0.175307,0.035248,0.004503,0.133903,0.293953,0.357087,1.220210
12945,326826639,0,0.0,2143537853,0,0.0,1203437299,35549,Other,0.0,...,15,Other,4,0.467905,0.071736,0.268770,0.009989,0.034089,0.147511,0.942951
12946,918950919,0,0.0,2145646585,0,0.0,3266555556,22147,Other,0.0,...,3,Other,4,0.192556,0.097234,0.571838,0.007002,0.019188,0.112183,2.187625


In [17]:
score_mean = ens_oof[(ens_oof["total_evaluators"] >= LB_EXPERTNUM_THR_LOW) & (ens_oof["total_evaluators"] <= LB_EXPERTNUM_THR_HIGH)]["score"].mean()
print(score_mean)

0.9151784331257757


In [24]:
ens_oof = model1_oof.copy()
pred_cols = [col for col in ens_oof.columns if "pred" in col]
print(pred_cols)
for col in pred_cols:
    ens_oof[col] = (model2_oof[col] + model3_oof[col]) / 2
ens_oof[pred_cols] = ens_oof[pred_cols] / ens_oof[pred_cols].values.sum(axis=1, keepdims=True)

['pred_seizure_vote', 'pred_lpd_vote', 'pred_gpd_vote', 'pred_lrda_vote', 'pred_grda_vote', 'pred_other_vote']


In [28]:
ens_oof

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,...,total_evaluators,target,fold,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote,score
0,1202099836,0,0.0,1353070,0,0.0,3051612434,34554,Other,0.0,...,14,Other,0,0.055458,0.059976,0.157455,0.033681,0.121492,0.571937,0.367644
1,1445780287,0,0.0,4004824,0,0.0,3042959589,22597,Other,0.0,...,17,Other,0,0.550663,0.112090,0.210461,0.011898,0.025287,0.089602,2.412380
2,3669581453,0,0.0,4546138,0,0.0,3616273655,57448,LRDA,0.0,...,3,LRDA,0,0.303979,0.043134,0.061732,0.015805,0.034121,0.541230,4.147447
3,3100586988,0,0.0,4960933,0,0.0,1900116647,35590,Other,0.0,...,2,Other,0,0.028856,0.045082,0.043460,0.024656,0.056355,0.801591,0.221156
4,2559567335,0,0.0,5487370,0,0.0,1418039992,36059,Other,0.0,...,12,Other,0,0.071127,0.024948,0.009986,0.090556,0.329151,0.474232,0.746059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12943,3570444920,0,0.0,2143352195,0,0.0,3891513135,31417,GRDA,0.0,...,3,GRDA,4,0.049894,0.059990,0.069197,0.138316,0.406295,0.276308,0.900675
12944,3897272486,0,0.0,2143410921,0,0.0,1360814571,63946,Other,0.0,...,15,Other,4,0.175307,0.035248,0.004503,0.133903,0.293953,0.357087,1.220210
12945,326826639,0,0.0,2143537853,0,0.0,1203437299,35549,Other,0.0,...,15,Other,4,0.467905,0.071736,0.268770,0.009989,0.034089,0.147511,0.942951
12946,918950919,0,0.0,2145646585,0,0.0,3266555556,22147,Other,0.0,...,3,Other,4,0.192556,0.097234,0.571838,0.007002,0.019188,0.112183,2.187625


In [25]:
ens_oof["score"] = ens_oof.progress_apply(compute_each_score, axis=1)

100%|██████████| 12948/12948 [02:02<00:00, 105.66it/s]


In [26]:
ens_oof["score"].mean()

1.8432250296532202

In [27]:
display(ens_oof.groupby("expert_consensus")["score"].mean())

expert_consensus
GPD        2.818633
GRDA       2.502811
LPD        2.262450
LRDA       2.437817
Other      1.096974
Seizure    2.097383
Name: score, dtype: float64